# LMNA


The data in this workbook were derived from Table 1 of [Merideth MA, et al (2008) Phenotype and course of Hutchinson-Gilford progeria syndrome. N Engl J Med. 358(6):592-604. PMID:18256394](https://pubmed.ncbi.nlm.nih.gov/18256394/).

The authors state: Every patient was heterozygous for the G608G mutation in LMNA
We mapped this to NM_170707.4(LMNA):c.1824C>T (p.Gly608=), see ClinVar VCV000014500.25

In [1]:
import pandas as pd
pd.set_option('display.max_colwidth', None) # show entire column contents, important!
from IPython.display import display, HTML
from pyphetools.creation import *
from pyphetools.visualization import *
from pyphetools.validation import *
import pyphetools
print(f"Using pyphetools version {pyphetools.__version__}")

Using pyphetools version 0.9.37


In [2]:
PMID = "PMID:18256394"
title = "Phenotype and course of Hutchinson-Gilford progeria syndrome"
cite = Citation(pmid=PMID, title=title)
parser = HpoParser(hpo_json_file="../hp.json")
hpo_cr = parser.get_hpo_concept_recognizer()
hpo_version = parser.get_version()
hpo_ontology = parser.get_ontology()
metadata = MetaData(created_by="ORCID:0000-0002-0736-9199", citation=cite)
metadata.default_versions_with_hpo(version=hpo_version)
print(f"HPO version {hpo_version}")

HPO version 2024-01-16


In [3]:
df = pd.read_csv("input/meridith2008.tsv", sep='\t')
df.head()

,Patient,Sex,Age,Lipodystrophy,Alopecia,FingertipTufting,AlteredSkinPigment,Circumoral cyanosis,ScleroticSkin,Hypodontia,...,systolic,diastolic,pulse,ECG,resting echo,intima right,intima left,ABI left,ABI right,neurological
0,1,F,"1,6",Yes,Yes,Yes,Yes,Yes,Yes,ND,...,108,76,103-150,Normal,Normal,0.29,0.28,ND,ND,NaN
1,2,M,"1,9",Yes,Yes,Yes,Yes,Yes,Yes,ND,...,123,78,115-143,Normal,Normal,ND,0.36,ND,ND,Increased intracranial pressure
2,3,F,"2,1",Yes,Yes,Yes,Yes,Yes,Yes,ND,...,108,60,118-129,Normal,Normal,0.39,0.47,ND,ND,NaN
3,4,F,"2,3",Yes,Yes,Yes,No,Yes,Yes,ND,...,136*,85*,125-142,"QT,463msec",Normal,0.32,0.32,1.17,1.37,NaN
4,5,F,"4,8",Yes,Yes,Yes,Yes,Yes,No,Yes,...,107,79*,95-119,Normal,Normal,0.46,0.36,1.08,1,NaN


In [4]:
generator = SimpleColumnMapperGenerator(df=df, observed="Yes", excluded="No", hpo_cr=hpo_cr)
column_mapper_list = generator.try_mapping_columns()
display(HTML(generator.to_html()))

Result,Columns
Mapped,Lipodystrophy; Alopecia; Circumoral cyanosis; Hypodontia; Ankyloglossia; Ogival palate; Delayed tooth eruption
Unmapped,Patient; Sex; Age; FingertipTufting; AlteredSkinPigment; ScleroticSkin; Double row of teeth; Sight; Glasses; IntraocularPressure; Corneal dryness; systolic; diastolic; pulse; ECG; resting echo; intima right; intima left; ABI left; ABI right; neurological


In [5]:
# "FingertipTufting" needs new HPO term

In [6]:
# Mottled pigmentation HP:0001070 (See figure 1)
pgMapper = SimpleColumnMapper(column_name="AlteredSkinPigment",
                              hpo_id="HP:0001070", hpo_label="Mottled pigmentation", observed="Yes", excluded="No")
column_mapper_list.append(pgMapper)
pgMapper.preview_column(df)

,mapping,count
0,"original value: ""Yes"" -> HP: Mottled pigmentation (HP:0001070) (observed)",14
1,"original value: ""No"" -> HP: Mottled pigmentation (HP:0001070) (excluded)",1


In [7]:
# The changes are scleroderma-like
# Scleroderma HP:0100324
skinMapper = SimpleColumnMapper(column_name="ScleroticSkin",hpo_id="HP:0100324", hpo_label="Scleroderma", observed="Yes", excluded="No")
column_mapper_list.append(skinMapper)
skinMapper.preview_column(df)

,mapping,count
0,"original value: ""Yes"" -> HP: Scleroderma (HP:0100324) (observed)",12
1,"original value: ""No"" -> HP: Scleroderma (HP:0100324) (excluded)",3


In [8]:
# Shark teeth HP:4000168
drtMapper = SimpleColumnMapper(column_name="Double row of teeth",hpo_id="HP:4000168", hpo_label="Shark teeth", observed="Yes", excluded="No")
column_mapper_list.append(drtMapper)
drtMapper.preview_column(df)

,mapping,count
0,"original value: ""No"" -> HP: Shark teeth (HP:4000168) (excluded)",12
1,"original value: ""Yes"" -> HP: Shark teeth (HP:4000168) (observed)",3


In [9]:
# only needed for creating of templates
#res = OptionColumnMapper.autoformat(df=df, concept_recognizer=hpo_cr)
#print(res)

In [10]:
sight_d = {
 'Hyperopic': 'Hypermetropia'}
excluded = { 'Normal': 'Abnormality of vision'}
sightMapper = OptionColumnMapper(column_name='Sight',concept_recognizer=hpo_cr, option_d=sight_d, excluded_d=excluded)
column_mapper_list.append(sightMapper)
sightMapper.preview_column(df)

,mapping,count
0,Hypermetropia (HP:0000540) (observed),8
1,Abnormality of vision (HP:0000504) (excluded),3


In [11]:
iop_d = {}
exclude = {'Normal': 'Abnormal intraocular pressure',}
iopMapper = OptionColumnMapper(column_name='IntraocularPressure',
                            concept_recognizer=hpo_cr, option_d=iop_d, excluded_d=exclude)
column_mapper_list.append(iopMapper)
iopMapper.preview_column(df)

,mapping,count
0,Abnormal intraocular pressure (HP:0012632) (excluded),8


In [12]:
# Corneal dryness --  is this different from Keratoconjunctivitis sicca HP:0001097 - unclear from description, omit

In [13]:
#  "systolic"; "diastolic";
# Seven children (Patients 4, 5, 6, 7, 10, 12, and 14)
# had systolic or diastolic blood pressures that were elevated 
# Elevated systolic blood pressure HP:0004421
# Elevated diastolic blood pressure HP:0005117

systolic_d = {
 '136*': 'Elevated systolic blood pressure',
 '128*': 'Elevated systolic blood pressure',
 '124*': 'Elevated systolic blood pressure',
 '169*': 'Elevated systolic blood pressure',}
systolicMapper = OptionColumnMapper(column_name='systolic',concept_recognizer=hpo_cr, option_d=systolic_d)
column_mapper_list.append(systolicMapper)
systolicMapper.preview_column(df)

,mapping,count
0,Elevated systolic blood pressure (HP:0004421) (observed),5


In [14]:
diastolic_d = {
 '85*': 'Elevated diastolic blood pressure',
 '79*': 'Elevated diastolic blood pressure',
 '64*': 'Elevated diastolic blood pressure',
 '100*': 'Elevated diastolic blood pressure'}
diastolicMapper = OptionColumnMapper(column_name='diastolic',concept_recognizer=hpo_cr, option_d=diastolic_d)
column_mapper_list.append(diastolicMapper)
diastolicMapper.preview_column(df)

,mapping,count
0,Elevated diastolic blood pressure (HP:0005117) (observed),4


In [15]:
# Prolonged QT interval HP:0001657
# normal is less than 440 msec
ecg_d = {
 '463msec': 'Prolonged QT interval',
 '458msec': 'Prolonged QT interval',
 '452msec': 'Prolonged QT interval',
 '456msec': 'Prolonged QT interval'}
ecgMapper = OptionColumnMapper(column_name='ECG',concept_recognizer=hpo_cr, option_d=ecg_d)
column_mapper_list.append(ecgMapper)
ecgMapper.preview_column(df)

,mapping,count
0,Prolonged QT interval (HP:0001657) (observed),5


In [16]:
neuro_d = {
 'Increased intracranial pressure': 'Increased intracranial pressure',
 'Carotid artery occlusion': 'Carotid artery occlusion',
 'Arteriosclerosis': 'Arteriosclerosis',
 'Transient ischemic attack': 'Transient ischemic attack',
 'Cerebral artery stenosis': 'Cerebral artery stenosis',
 'Carotid artery stenosis': 'Carotid artery stenosis'}
neurologicalMapper = OptionColumnMapper(column_name='neurological',concept_recognizer=hpo_cr, option_d=neuro_d)
column_mapper_list.append(neurologicalMapper)
neurologicalMapper.preview_column(df)

,mapping,count
0,Increased intracranial pressure (HP:0002516) (observed),1
1,Carotid artery occlusion (HP:0012474) (observed),1
2,Arteriosclerosis (HP:0002634) (observed),1
3,Transient ischemic attack (HP:0002326) (observed),1
4,Cerebral artery stenosis (HP:0012492) (observed),1
5,Carotid artery stenosis (HP:0100546) (observed),1


In [17]:
sexMapper = SexColumnMapper(male_symbol="M", female_symbol="F", column_name="Sex")
#sexMapper.preview_column(df)

In [18]:
def to_iso(year_month):
    fields = year_month.split(",")
    if len(fields) != 2:
        raise ValueError(f"Malformed age {year_month}")
    y = fields[0]
    m = fields[1]
    return f"P{y}Y{m}M"
df['iso_age'] = df["Age"].apply(lambda x: to_iso(x))
ageMapper = AgeColumnMapper.iso8601(column_name='iso_age')
#ageMapper.preview_column(df)

In [19]:
LMNA_transcript = "NM_170707.4"
vvalidator = VariantValidator(genome_build="hg38", transcript=LMNA_transcript)
# heterozygosity in affected members for a 1-bp deletion (del959T) deletion in exon 6 of the LMNA gene
# NM_170707.4(LMNA):c.960del (p.Arg321fs) 
var = vvalidator.encode_hgvs("c.1824C>T")
var.set_heterozygous()

https://rest.variantvalidator.org/VariantValidator/variantvalidator/hg38/NM_170707.4%3Ac.1824C>T/NM_170707.4?content-type=application%2Fjson


In [20]:
encoder = CohortEncoder(df=df, 
                        hpo_cr=hpo_cr, 
                        column_mapper_list=column_mapper_list, 
                        individual_column_name="Patient", 
                        agemapper=ageMapper, 
                        sexmapper=sexMapper,
                        metadata=metadata)
omim_id = "OMIM:176670"
omim_label = "Hutchinson-Gilford progeria"
 	 
cmp1a = Disease(disease_id=omim_id, disease_label=omim_label)
encoder.set_disease(disease=cmp1a)

In [21]:
individuals = encoder.get_individuals()
for i in individuals:
    i.add_variant(var)

In [22]:
## All 15 children appeared aged and had 
## prominent eyes  = Proptosis HP:0000520
## micrognathia = Micrognathia HP:0000347
## decreased subcutaneous fat = Reduced subcutaneous adipose tissue HP:0003758
## alopecia, = Alopecia HP:0001596
## skin dimpling and mottling, 
## prominent cutaneous vasculature, = Prominent superficial blood vessels HP:0007394
## fingertip tufting, and distal-joint abnormalities
## Mean weight, which was normal at birth for all 15 children, decreased below the third percentile by 2 months of age 
## Failure to thrive HP:0001508
## Height for children with Hutchinson–Gilford progeria syndrome decreased below the third percentile for normal height by 15 months of age 
## Short stature HP:0004322
## Bone mineral density of the lumbar spine was decreased in all 14 children tested (assume true for all, since details not given)
## Reduced bone mineral density HP:0004349
for i in individuals:
    proptosis = HpTerm(hpo_id="HP:0000520", label="Proptosis")
    i.add_hpo_term(proptosis)
    micrognathia = HpTerm(hpo_id="HP:0000347", label="Micrognathia")
    i.add_hpo_term(micrognathia)
    sqfat = HpTerm(hpo_id="HP:0003758", label="Reduced subcutaneous adipose tissue")
    i.add_hpo_term(sqfat)
    alopecia = HpTerm(hpo_id="HP:0001596", label="Alopecia")
    i.add_hpo_term(alopecia)
    vessels = HpTerm(hpo_id="HP:0007394", label="Prominent superficial blood vessels")
    i.add_hpo_term(vessels)
    ftt = HpTerm(hpo_id="HP:0001508", label="Failure to thrive")
    i.add_hpo_term(ftt)
    sst = HpTerm(hpo_id="HP:0004322", label="Short stature")
    i.add_hpo_term(sst)
    bmt = HpTerm(hpo_id="HP:0004349", label="Reduced bone mineral density")
    i.add_hpo_term(bmt)

In [23]:
## Radiologic examinations showed acro-osteolysis = Osteolytic defects of the phalanges of the hand HP:0009771
## clavicular resorption, = Progressive clavicular acroosteolysis HP:0000905
## and coxa valga = Coxa valga HP:0002673
## in all 15 children
for i in individuals:
    acroost = HpTerm(hpo_id="HP:0009771", label="Osteolytic defects of the phalanges of the hand")
    i.add_hpo_term(acroost)
    cvalga = HpTerm(hpo_id="HP:0002673", label="Coxa valga")
    i.add_hpo_term(cvalga)
    cr = HpTerm(hpo_id="HP:0000905", label="Progressive clavicular acroosteolysis")
    i.add_hpo_term(cr)

In [24]:
cvalidator = CohortValidator(cohort=individuals, ontology=hpo_ontology, min_hpo=1, allelic_requirement=AllelicRequirement.MONO_ALLELIC)
qc = QcVisualizer(cohort_validator=cvalidator)
display(HTML(qc.to_summary_html()))

Level,Error category,Count
INFORMATION,NOT_MEASURED,5


In [25]:
individuals = cvalidator.get_error_free_individual_list()
table = PhenopacketTable(individual_list=individuals, metadata=metadata)
display(HTML(table.to_html()))

Individual,Disease,Genotype,Phenotypic features
1 (FEMALE; P1Y6M),Hutchinson-Gilford progeria (OMIM:176670),NM_170707.4:c.1824C>T (heterozygous),Prominent superficial blood vessels (HP:0007394); Failure to thrive (HP:0001508); Osteolytic defects of the phalanges of the hand (HP:0009771); Micrognathia (HP:0000347); Mottled pigmentation (HP:0001070); Progressive clavicular acroosteolysis (HP:0000905); Reduced subcutaneous adipose tissue (HP:0003758); Lipodystrophy (HP:0009125); Alopecia (HP:0001596); Short stature (HP:0004322); Reduced bone mineral density (HP:0004349); Scleroderma (HP:0100324); Proptosis (HP:0000520); Coxa valga (HP:0002673); Delayed eruption of teeth (HP:0000684); Circumoral cyanosis (HP:0032556); excluded: Ankyloglossia (HP:0010296); excluded: High palate (HP:0000218); excluded: Shark teeth (HP:4000168); excluded: Abnormal intraocular pressure (HP:0012632)
2 (MALE; P1Y9M),Hutchinson-Gilford progeria (OMIM:176670),NM_170707.4:c.1824C>T (heterozygous),Reduced bone mineral density (HP:0004349); Scleroderma (HP:0100324); Ankyloglossia (HP:0010296); Prominent superficial blood vessels (HP:0007394); Failure to thrive (HP:0001508); Alopecia (HP:0001596); Short stature (HP:0004322); Increased intracranial pressure (HP:0002516); Osteolytic defects of the phalanges of the hand (HP:0009771); Reduced subcutaneous adipose tissue (HP:0003758); Proptosis (HP:0000520); Coxa valga (HP:0002673); Micrognathia (HP:0000347); Mottled pigmentation (HP:0001070); Progressive clavicular acroosteolysis (HP:0000905); Hypermetropia (HP:0000540); Lipodystrophy (HP:0009125); Delayed eruption of teeth (HP:0000684); Circumoral cyanosis (HP:0032556); excluded: High palate (HP:0000218); excluded: Shark teeth (HP:4000168)
3 (FEMALE; P2Y1M),Hutchinson-Gilford progeria (OMIM:176670),NM_170707.4:c.1824C>T (heterozygous),Prominent superficial blood vessels (HP:0007394); Failure to thrive (HP:0001508); Osteolytic defects of the phalanges of the hand (HP:0009771); Micrognathia (HP:0000347); Mottled pigmentation (HP:0001070); Progressive clavicular acroosteolysis (HP:0000905); Reduced subcutaneous adipose tissue (HP:0003758); Lipodystrophy (HP:0009125); Alopecia (HP:0001596); Short stature (HP:0004322); Reduced bone mineral density (HP:0004349); Scleroderma (HP:0100324); Proptosis (HP:0000520); Coxa valga (HP:0002673); Delayed eruption of teeth (HP:0000684); Circumoral cyanosis (HP:0032556); excluded: Ankyloglossia (HP:0010296); excluded: High palate (HP:0000218); excluded: Shark teeth (HP:4000168); excluded: Abnormality of vision (HP:0000504); excluded: Abnormal intraocular pressure (HP:0012632)
4 (FEMALE; P2Y3M),Hutchinson-Gilford progeria (OMIM:176670),NM_170707.4:c.1824C>T (heterozygous),High palate (HP:0000218); Elevated diastolic blood pressure (HP:0005117); Reduced bone mineral density (HP:0004349); Scleroderma (HP:0100324); Ankyloglossia (HP:0010296); Prominent superficial blood vessels (HP:0007394); Failure to thrive (HP:0001508); Alopecia (HP:0001596); Short stature (HP:0004322); Prolonged QT interval (HP:0001657); Osteolytic defects of the phalanges of the hand (HP:0009771); Reduced subcutaneous adipose tissue (HP:0003758); Elevated systolic blood pressure (HP:0004421); Proptosis (HP:0000520); Coxa valga (HP:0002673); Micrognathia (HP:0000347); Progressive clavicular acroosteolysis (HP:0000905); Lipodystrophy (HP:0009125); Delayed eruption of teeth (HP:0000684); Circumoral cyanosis (HP:0032556); excluded: Mottled pigmentation (HP:0001070); excluded: Shark teeth (HP:4000168); excluded: Abnormality of vision (HP:0000504)
5 (FEMALE; P4Y8M),Hutchinson-Gilford progeria (OMIM:176670),NM_170707.4:c.1824C>T (heterozygous),Prominent superficial blood vessels (HP:0007394); Failure to thrive (HP:0001508); Osteolytic defects of the phalanges of the hand (HP:0009771); Micrognathia (HP:0000347); Mottled pigmentation (HP:0001070); Progressive clavicular acroosteolysis (HP:0000905); Hypermetropia (HP:0000540); Elevated diastolic blood pressure (HP:0005117)

In [26]:
Individual.output_individuals_as_phenopackets(individual_list=individuals, metadata=metadata)

We output 15 GA4GH phenopackets to the directory phenopackets
